(ex-electronic-medical-record)=
# (EX) Electronic medical record
This example utilizes the [MIMIC III demo](https://physionet.org/content/mimiciii-demo/1.4/) dataset in illustrating the use of database using DuckDB.

*Quick notes about DuckDB:*  

- DuckDB is a relational database for analytics processing (i.e., OLAP)
- DuckDB is columnar-oriented
- DuckDB scales reasonably for *relatively* large datasets and works well for local development

In [57]:
import duckdb

In [63]:
# establish connection
conn = duckdb.connect('../lab/dataset/mimic.db', read_only=True)

conn.sql('SHOW TABLES;')

In [59]:
# metadata for add
conn.sql(
    """
    SELECT column_name, data_type FROM information_schema.columns 
    WHERE table_name = 'ICDPROCS';
    """
)

┌─────────────┬───────────┐
│ column_name │ data_type │
│   varchar   │  varchar  │
├─────────────┼───────────┤
│ row_id      │ INTEGER   │
│ icd9_code   │ INTEGER   │
│ short_title │ VARCHAR   │
│ long_title  │ VARCHAR   │
└─────────────┴───────────┘

In [55]:
# admission table at a glance
conn.sql(
    """
    
    """
)

┌────────┬────────────┬─────────┬─────────────────────┬───┬─────────────────────┬─────────────────────┬─────────────┐
│ row_id │ subject_id │ gender  │         dob         │ … │      dod_hosp       │       dod_ssn       │ expire_flag │
│ int32  │   int32    │ varchar │       varchar       │   │       varchar       │       varchar       │    int32    │
├────────┼────────────┼─────────┼─────────────────────┼───┼─────────────────────┼─────────────────────┼─────────────┤
│   9467 │      10006 │ F       │ 2094-03-05 00:00:00 │ … │ 2165-08-12 00:00:00 │ 2165-08-12 00:00:00 │           1 │
│   9472 │      10011 │ F       │ 2090-06-05 00:00:00 │ … │ 2126-08-28 00:00:00 │ NULL                │           1 │
│   9474 │      10013 │ F       │ 2038-09-03 00:00:00 │ … │ 2125-10-07 00:00:00 │ 2125-10-07 00:00:00 │           1 │
│   9478 │      10017 │ F       │ 2075-09-21 00:00:00 │ … │ NULL                │ 2152-09-12 00:00:00 │           1 │
│   9479 │      10019 │ M       │ 2114-06-20 00:00:00 │ 

## Simple queries

In [47]:
# number of patients
conn.sql(
    """

    """
)

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│          100 │
└──────────────┘

In [ ]:
# number of patients by gender
conn.sql(
    """

    """
)

In [ ]:
# take a glance at the ICUSTAY table
conn.sql(
    """

    """
)

## Derivative queries

In [ ]:
# calculate the age of each patient and include in the table above
conn.sql(
    """

    """
)

In [ ]:
# calculate the duration of admission before each patient is admitted to the ICU
conn.sql(
    """

    """
)

In [ ]:
# identify if a patient passes away when they are in the ICU
conn.sql(
    """

    """
)

## Additional reference queries

In [56]:
# 
conn.sql(
    """
    SELECT p.subject_id, p.dob, a.hadm_id,
        a.admittime, p.expire_flag
    FROM admissions a
    INNER JOIN patients p
    ON p.subject_id = a.subject_id
    ORDER BY admittime;
    """
)

In [51]:
 # 
conn.sql(
    """
    SELECT p.subject_id, p.dob, a.hadm_id,
        a.admittime, p.expire_flag,
        MIN (a.admittime) OVER (PARTITION BY p.subject_id) AS first_admittime
    FROM admissions a
    INNER JOIN patients p
    ON p.subject_id = a.subject_id
    ORDER BY a.hadm_id, p.subject_id;
    """
)

┌────────────┬─────────────────────┬─────────┬─────────────────────┬─────────────┬─────────────────────┐
│ subject_id │         dob         │ hadm_id │      admittime      │ expire_flag │   first_admittime   │
│   int32    │       varchar       │  int32  │       varchar       │    int32    │       varchar       │
├────────────┼─────────────────────┼─────────┼─────────────────────┼─────────────┼─────────────────────┤
│      10056 │ 2046-02-27 00:00:00 │  100375 │ 2129-05-02 00:12:00 │           1 │ 2129-05-02 00:12:00 │
│      42430 │ 2061-12-10 00:00:00 │  100969 │ 2142-11-26 21:20:00 │           1 │ 2142-11-26 21:20:00 │
│      41914 │ 2090-11-16 00:00:00 │  101361 │ 2145-12-01 18:13:00 │           1 │ 2145-12-01 18:13:00 │
│      42135 │ 2083-09-20 00:00:00 │  102203 │ 2127-07-23 15:21:00 │           1 │ 2127-07-23 15:21:00 │
│      44228 │ 2112-10-22 00:00:00 │  103379 │ 2170-12-15 03:14:00 │           1 │ 2170-12-15 03:14:00 │
│      10026 │ 1895-05-17 00:00:00 │  103770 │ 2195-05-